In [55]:
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if (!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, httr,jsonlite)

In [56]:
##Hacer lista con urls de los archivos

presupuestos <- list(
  '2024' = 'https://datos.cdmx.gob.mx/dataset/3a6cba97-adb1-4f35-9df1-984cc3657aca/resource/7fa1da81-531c-477c-93db-7580bc1c7ad0/download/egresos_2024_aprobado.csv',
  '2023' = 'https://datos.cdmx.gob.mx/dataset/ea119a03-4f5a-46fa-afbe-bc32bcf79ec3/resource/5137e3db-227e-462d-8d59-1942afbc0c6a/download/egresos_2023_aprobado.csv',
  '2022' = 'https://datos.cdmx.gob.mx/dataset/755f47f1-b937-4131-8a7b-fbe306e747c8/resource/fc93c940-ad75-4e7f-afe6-c624726623a9/download/egresos_2022_aprobado.csv',
  '2021' = 'https://datos.cdmx.gob.mx/dataset/246be014-62ba-46e0-a7ce-9f37c8f01886/resource/682c37bd-96a2-4f78-94bf-3c77c3870f04/download/egresos_2021_aprobado.csv',
  '2020' = 'https://datos.cdmx.gob.mx/dataset/2bb4de63-8212-4d2a-a377-98b10f8384c6/resource/def0fc56-4e26-4cd2-9ec6-200cb86b418e/download/egresos_2020_aprobado.csv',
  '2019' = 'https://datos.cdmx.gob.mx/dataset/8dc52617-63f6-4bfd-9fae-54239675c6d0/resource/9a6f53d7-1751-4ae4-8bd8-d31807be7789/download/egresos_2019_aprobado.csv',
  '2018' = 'https://datos.cdmx.gob.mx/dataset/7a8820f2-3bea-4c69-8b20-1d01b1a99a6a/resource/5b20655c-072f-4347-baee-ab93d930b27b/download/egresos_2018_aprobado.csv'
)

url_ipc<-"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/910392/es/0700/false/BIE/2.0/fe198fe7-c264-840e-c24e-7b28b9af856c?type=json"

In [57]:
#Función para consultar la api

consulta_api<-function(url){
  r <- GET(url, timeout(10000))
  df <- fromJSON(rawToChar(r$content))
  return(df)
}

In [58]:
ipccdmx<-consulta_api(url_ipc)%>%
pluck("Series")%>%
pluck("OBSERVATIONS")%>%
as.data.frame()%>%
janitor::clean_names()%>%
select(time_period, obs_value)%>%
rename(
ipc=obs_value)%>%
#Extraer el año de time_period
mutate(year=str_extract(time_period, "\\d{4}"))%>%
mutate(year=as.numeric(year),
ipc=as.numeric(ipc))%>%
filter(year>=2018 & year<=2024)%>%
arrange(year)%>%
#Hacer un promedio de los valores de ipc por año
group_by(year)%>%
summarise(ipc=mean(ipc, na.rm=TRUE))%>%
#Dividir todo entre el valor de 2024
mutate(ipc=ipc/last(ipc)*100)
ipccdmx

year,ipc
<dbl>,<dbl>
2018,74.05232
2019,76.74483
2020,79.35172
2021,83.86621
2022,90.48852
2023,95.49069
2024,100.00000


In [59]:
presupuestos <- map_dfr(
  names(presupuestos),
  ~ read_csv(presupuestos[[.x]], show_col_types = FALSE) %>%
    mutate(year = as.numeric(.x))
) %>%
  # Identificar presupuesto de obra pública
  mutate(obra_pub = case_when(capitulo == 6000 ~ 1, TRUE ~ 0)) %>%
  group_by(year) %>%
  summarise(
    presup_opc = sum(monto_aprobado * obra_pub, na.rm = TRUE),
    presup_total = sum(monto_aprobado, na.rm = TRUE)
  ) %>%
  #Añadir 2025. El dato viene de: https://cdmxassets.s3.us-east-1.amazonaws.com/media/files-pdf/paquete_economico_2025/ANEXO_III/IIIA_C_objeto_gasto.pdf
  #add_row(year = 2025, presup_opc = 21535048926.50, presup_total = 291525557665.00) %>%
  arrange(year)%>%
  mutate(pct= presup_opc / presup_total * 100) %>%
  left_join(ipccdmx, by = "year")%>%
  #Calcular presupuesto de obra pública en términos reales
  mutate(presup_opc_real = presup_opc / (ipc / 100)) %>%
  mutate(presup_total_real = presup_total / (ipc / 100)) %>%
  mutate(part_real = presup_opc_real / presup_total_real * 100)%>%
  #Sacar variación porcentual del presupuesto de obra pública en términos reales
  mutate(
    variacion_real = (presup_opc_real - lag(presup_opc_real)) / lag(presup_opc_real) * 100,
    variacion_nominal = (presup_opc - lag(presup_opc)) / lag(presup_opc) * 100
  ) 
presupuestos


year,presup_opc,presup_total,pct,ipc,presup_opc_real,presup_total_real,part_real,variacion_real,variacion_nominal
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018,24241841910,226851897840,10.686198,74.05232,32736102589,306340047414,10.686198,NA,NA
2019,34280506505,234016325579,14.648767,76.74483,44668164686,304927809880,14.648767,36.449245,41.410486
2020,30158788208,238974793216,12.620071,79.35172,38006469100,301158920290,12.620071,-14.913744,-12.023505
2021,19729252363,217962157170,9.051687,83.86621,23524674315,259892704802,9.051687,-38.103500,-34.582079
2022,20362718481,234005875723,8.701798,90.48852,22503097011,258602844579,8.701798,-4.342578,3.210796
2023,18824794269,248415083383,7.577959,95.49069,19713749224,260145879276,7.577959,-12.395395,-7.552647
2024,18148035540,267965350437,6.772531,100.00000,18148035540,267965350437,6.772531,-7.942242,-3.595039
